In [ ]:
#import torch packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import numpy as np
from scipy.io import loadmat,savemat
import matplotlib.pyplot as plt

In [ ]:
#for docomo, 9 CUDAs are available in total, we only use CUDA 0 is this program
num_device = torch.cuda.device_count()
print('There are '+str(num_device)+' GPUs. We will use CUDA 0 in this program.')
device = torch.device('cuda:0')

In [ ]:
#assign parameter values
input_nodes=512
output_nodes=512
middle_nodes=512
test_rate=0.2
max_epoch=100
lr=1e-3
dropout_rate=0.05
num_BS=4
num_beam=512

In [ ]:
#load datasets from .mat files
in_set_file = loadmat('DLCB_Dataset/DLCB_input.mat')
in_set = in_set_file['DL_input']#in_set.shape=(54481,256)
out_set_file = loadmat('DLCB_Dataset/DLCB_output.mat')
out_set = out_set_file['DL_output']#out_set.shape=(54481,2048)

In [ ]:
#define a Network
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_nodes,middle_nodes)
        self.fc2 = nn.Linear(middle_nodes,middle_nodes)
        self.fc3 = nn.Linear(middle_nodes,middle_nodes)
        self.fc4 = nn.Linear(middle_nodes,middle_nodes)
        self.fc5 = nn.Linear(middle_nodes,output_nodes)
        
    def forward(self,x):
        # 4 hidden layers with each fc + relu + dropout
        x = x.to(device)
        x = x.view(-1,512)
        x = F.dropout(F.relu(self.fc1(x)),p = dropout_rate,training = True)
        x = F.dropout(F.relu(self.fc2(x)),p = dropout_rate,training = True)
        x = F.dropout(F.relu(self.fc3(x)),p = dropout_rate,training = True)
        x = F.dropout(F.relu(self.fc4(x)),p = dropout_rate,training = True)
        # 1 output layer with fc + relu
        x = F.relu(self.fc5(x))
        return x  
net = Net()
net = net.to(device)

#define Loss Function and Optimization method
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(net.parameters(),lr=lr)

In [ ]:
class DataSetGenerator(data.Dataset):
    def __init__(self,input_set,output_set):
        self.input_set = input_set
        self.output_set = output_set
     
    def __getitem__(self,index):
        x=[]
        for j in range(self.input_set.shape[1]):
            x.append(self.input_set[index][j])
        label=[]
        for j in range(self.output_set.shape[1]):
            label.append(self.output_set[index][j])
        return torch.FloatTensor(x),torch.FloatTensor(label)

    def __len__(self):
        return self.input_set.shape[0]

In [ ]:
#get num_total_user from in_set
def getTotalUser(in_set):
    num_total_user=in_set.shape[0]
    ##print('Number of total users in input dataset is '+str(num_total_user))
    return num_total_user
num_total_user = getTotalUser(in_set)

In [ ]:
#get DL_size_list, 17 percentages are fixed according to the author
def getDLSizeList():
    DL_pr_list = [.001,.05,.1,.15,.2,.25,.3,.35,.4,.45,.5,.55,.6,.65,.7,.75,.8]
    DL_size_list = []
    for i in range(len(DL_pr_list)):
        DL_size_list.append(int(DL_pr_list[i]*num_total_user))
    ##print('Number of users in DL is '+str(DL_size_list))
    return DL_size_list
DL_size_list = getDLSizeList()

In [ ]:
#get num_train and num_test, both are lists according to DL_size_list, num_total_user, test_rate
def getAmount(DL_size_list,num_total_user,test_rate):
    num_train=[]
    num_test=[]
    for i in range(len(DL_size_list)):
        num_train.append(int((DL_size_list[i])*0.8))
        num_test.append(int(num_total_user*test_rate))
    ##print(str(num_train)+' training examples.\n'+str(num_test)+' test examples.')
    return num_train,num_test
num_train,num_test = getAmount(DL_size_list,num_total_user,test_rate)
##print(len(num_train)==len(num_test))

In [ ]:
#specifically for the i-th DL_size, return its in_train_pre,in_test_pre,out_train_pre,out_test_pre(all before any adjustments)
def eachInOutPre(i,num_train,num_test,num_total_user,in_set,out_set):
    train_index = np.random.choice(range(0,num_total_user),size=num_train[i],replace=False)
    rem_index = set(range(0,num_total_user))-set(train_index)
    test_index = list(set(np.random.choice(list(rem_index),size=num_test[i],replace=False)))
    in_train_pre = in_set[train_index]
    in_test_pre = in_set[test_index]
    out_train_pre = out_set[train_index]
    out_test_pre = out_set[test_index]
    return in_train_pre,out_train_pre,in_test_pre,out_test_pre,test_index
##in_train_pre,out_train_pre,in_test_pre,out_test_pre,test_index = eachInOutPre(2,num_train,num_test,num_total_user,in_set,out_set)
##print(in_train_pre.shape,out_train_pre.shape)

In [ ]:
#arrange input into real part and imaginary part one by one
def arrangeInput(lst):
    order = np.zeros((lst.shape[0],2*lst.shape[1]))
    for i in range(lst.shape[0]):
        for j in range(lst.shape[1]):
            order[i][2*j] = np.float64(lst[i][j].real)
            order[i][2*j+1] = np.float64(lst[i][j].imag)
    return order
##in_train = arrangeInput(in_train_pre)
##in_test = arrangeInput(in_test_pre)
##print(in_train_pre[12][85],in_train[12][85*2],in_train[12][85*2+1])

In [ ]:
#specificially for i BS(s) in usage, divide output columns by num_beam
def divideOutput(i,num_BS,num_beam,lst):
    out = np.zeros((lst.shape[0],int(lst.shape[1]*(1/num_BS))))
    for j in range(lst.shape[0]):
        for k in range(int(lst.shape[1]*(1/num_BS))):
            out[j][k] = lst[j][k+i*num_beam]
    return out
##out_train = divideOutput(2,num_BS,num_beam,out_train_pre)
##out_test = divideOutput(2,num_BS,num_beam,out_test_pre)
##print(out_train_pre.shape[1]==2048,out_train.shape[1]==512)
##print(out_train_pre[12][85+1024],out_train[12][85])

In [ ]:
#for the i_DL-th DL_size and i_Out BS(s), generate and load datasets, where i_DL 0-16, i_Out 0-3
def getLoader(i_DL,i_Out):
    in_train_pre,out_train_pre,in_test_pre,out_test_pre,test_index = eachInOutPre(i_DL,num_train,num_test,num_total_user,in_set,out_set)
    in_train = arrangeInput(in_train_pre)
    in_test = arrangeInput(in_test_pre)
    out_train = divideOutput(i_Out,num_BS,num_beam,out_train_pre)
    out_test = divideOutput(i_Out,num_BS,num_beam,out_test_pre)

    train_generator = DataSetGenerator(input_set = in_train, output_set = out_train)
    train_loader = data.DataLoader(dataset=train_generator, batch_size=100,shuffle=True)
    
    test_generator = DataSetGenerator(input_set = in_test, output_set = out_test)
    test_loader = data.DataLoader(dataset=test_generator, batch_size=10,shuffle=True)
    return train_loader,test_loader,in_test,out_test
##train_loader,test_loader,in_test,out_test = getLoader(12,2)

In [ ]:
#every 50 epochs, print a loss statistics, flag is set to be 0 if it is a training set, otherwise 1
def printS(i_DL,i_Out,epoch,loss,flag):
    if epoch%50==49:
        DL_size_list = getDLSizeList()
        num_train,num_test = getAmount(DL_size_list,num_total_user,test_rate)
        if flag==0:
            print('Current DL size is {}, including {} BS.\nCurrent Training Epoch is {}, Loss is {:.8f}'.format(
                DL_size_list[i_DL],i_Out+1,epoch+1,sum(loss)/len(loss)))
        if flag==1:
            print('Current DL size is {}, including {} BS.\nCurrent Testing Epoch is {}, Loss is {:.8f}'.format(
                DL_size_list[i_DL],i_Out+1,epoch+1,sum(loss)/len(loss)))

In [ ]:
#train the network
for i_DL in range(len(DL_size_list)):
    DL_Result = {}
    for i_Out in range(0,num_BS,1):
        train_loader,test_loader,in_test,out_test = getLoader(i_DL,i_Out) 
        for epoch in range(max_epoch):
            train_loss = []
            for batch_idx,(x,label) in enumerate(train_loader):
                x = x.to(device)
                label = label.to(device)
                optimizer.zero_grad()
                y_hat = net(x)
                loss = criterion(label.to(torch.float32),y_hat.to(torch.float32))
                train_loss.append(loss)
                loss.backward()
                optimizer.step()
            printS(i_DL,i_Out,epoch,train_loss,0)
            test_loss = []
            with torch.no_grad():
                for batch_idx,(x,label) in enumerate(test_loader):
                    x = x.to(device)
                    label = label.to(device)
                    optimizer.zero_grad()
                    y_hat = net(x)
                    loss = criterion(label.to(torch.float32),y_hat.to(torch.float32))
                    test_loss.append(loss)
            printS(i_DL,i_Out,epoch,test_loss,1)
        if epoch==(max_epoch-1):
            print('End Train and Test of DL size {} and BS {} successfully.'.format(DL_size_list[i_DL],i_Out+1))
        
        #store DL_Result and save in .mat file
        DL_Result['TX'+str(i_Out+1)+'Pred_Beams'] = y_hat
        DL_Result['TX'+str(i_Out+1)+'Opt_Beams'] = out_test[:,:]
    _,_,_,_,test_index = eachInOutPre(i_DL,num_train,num_test,num_total_user,in_set,out_set)
    DL_Result['user_index'] = test_index
    savemat('DLCB_code_output/DL_Result'+str(i_DL+1),DL_Result)

In [ ]:
#save model to path
path = './DLCB_code_output/model_train'+str(len(in_train))+'_test'+str(len(in_test))+'_epoch'+str(max_epoch)+'.pth'
torch.save(net.state_dict(),path)